In [1]:
import pandas as pd
import json
import collections
import nltk as 

In [2]:
#Load in the tweet data, append it line by line to an empty list
tweets_data = []
tweets_file = open('scrape.txt', 'r')
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [3]:
#The number of tweets in my list
len(tweets_data)

541

In [27]:
tweets_df = pd.DataFrame()

In [8]:
def ignore_keyerrors(tweet):
    try:
        tweet['user']['location']
    except:
        return None
    return tweet['user']['location']

In [13]:
ignore_keyerrors(tweets_data[3])

'Macapá-Amapá'

In [31]:
tweets_df['location'] = [ignore_keyerrors(tweet) for tweet in tweets_data]

In [32]:
tweets_df

,location
0,None
1,"Nuevo León, México"
2,None
3,Macapá-Amapá
4,1d zm tvd to tflash
5,"Mafra, Lisboa, Portugal"
6,None
7,Peru
8,HonneyMoon Avenue
9,Harry's Fiance ♡


In [52]:
tweets_data[4]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Nov 24 21:52:17 +0000 2015',
 'entities': {'hashtags': [{'indices': [108, 116], 'text': 'GRAMMYs'}],
  'media': [{'display_url': 'pic.twitter.com/K6TBdvI0XO',
    'expanded_url': 'http://twitter.com/zouisclarity/status/669215683083952128/photo/1',
    'id': 669215675173466112,
    'id_str': '669215675173466112',
    'indices': [117, 140],
    'media_url': 'http://pbs.twimg.com/media/CUmIHGTWcAAelUG.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CUmIHGTWcAAelUG.jpg',
    'sizes': {'large': {'h': 419, 'resize': 'fit', 'w': 599},
     'medium': {'h': 419, 'resize': 'fit', 'w': 599},
     'small': {'h': 237, 'resize': 'fit', 'w': 340},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 669215683083952128,
    'source_status_id_str': '669215683083952128',
    'source_user_id': 795014636,
    'source_user_id_str': '795014636',
    'type': 'photo',
    'url': 'https://t.co/K6TBdvI0XO'}],
  

In [45]:
def has_geo(tweet):
    try:
        tweet['geo']
    except:
        return False
    if tweet['geo'] is not None: 
        return tweet['geo']
    else:
        return False

In [46]:
tweets_df['geo'] = [has_geo(tweet) for tweet in tweets_data]

In [48]:
tweets_df[tweets_df['geo']]

,location,geo


In [6]:
num = 0
for tweet in tweets_data:
        try:
            tweet['user']['location']
        except KeyError:
            num += 1
print(num)
            
    
        

541


In [53]:
def has_coordinates(tweet):
    try:
        tweet['coordinates']
    except:
        return False
    if tweet['coordinates'] is not None: 
        return tweet['coordinates']
    else:
        return False

In [54]:
tweets_df['coordinates'] = [has_coordinates(tweet) for tweet in tweets_data]

In [56]:
tweets_df[tweets_df['coordinates']]

,location,geo,coordinates


In [57]:
yum_data = []
yum_file = open('yum.txt', 'r')
for line in yum_file:
    try:
        tweet = json.loads(line)
        yum_data.append(tweet)
    except:
        continue

In [60]:
yum_data[2]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Nov 25 18:26:27 +0000 2015',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/cambriawines/s…',
    'expanded_url': 'https://twitter.com/cambriawines/status/667484488415485953',
    'indices': [88, 111],
    'url': 'https://t.co/Ml16jkDbb9'}],
  'user_mentions': [{'id': 87989672,
    'id_str': '87989672',
    'indices': [73, 86],
    'name': 'Cambria Wines',
    'screen_name': 'CambriaWines'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 669582936719642626,
 'id_str': '669582936719642626',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': True,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'quoted_status': {'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Nov 19 23:27:57 +00

In [61]:
yum_df = pd.DataFrame()

In [62]:
yum_df['text'] = [tweet['text'] for tweet in yum_data]

In [63]:
yum_df

,text
0,#butterbeer #diagonalley #leakycauldron #lunch...
1,Breakfast and Thanksgiving prep! #cook #cateri...
2,This recipe for cranberry sauce with a twist l...
3,I love Rosa's Meat Pasta #Yum #Myfavourite #Be...
4,RT @travelwithnick: Just saying... This is #ve...
5,My Dream that was beyond reaching &amp; no dir...
6,Thanks for joining us for another Foodtruck Fu...
7,The VERY young (and very SCRUMMY) @NyleDiMarco...
8,Hello #Amsterdam ! Ill soon be back to #blog a...
9,RT @MyAtkins: #WinWednesday RT &amp; follow to...


In [66]:
def open_and_load_tweets(file_name):
    data = []
    data_file = open(file_name, 'r')
    for line in data_file:
        try:
            tweet = json.loads(line)
            data.append(tweet)
        except:
            continue
    return data

In [73]:
thanksgiving = open_and_load_tweets('thanksgiving2.txt')

In [76]:
def has_text(tweet):
    try:
        tweet['text']
    except:
        return False
    if tweet['text'] is not None: 
        return tweet['text']
    else:
        return False

In [77]:
thanksgiving_df = pd.DataFrame()
thanksgiving_df['text'] = [has_text(tweet) for tweet in thanksgiving]

In [72]:
thanksgiving[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Nov 25 18:38:56 +0000 2015',
 'entities': {'hashtags': [{'indices': [124, 137], 'text': 'Thanksgiving'}],
  'media': [{'display_url': 'pic.twitter.com/o25MhbsZcj',
    'expanded_url': 'http://twitter.com/sleonardFlightV/status/669550509309583360/photo/1',
    'id': 669550508642725888,
    'id_str': '669550508642725888',
    'indices': [139, 140],
    'media_url': 'http://pbs.twimg.com/media/CUq4o-LU8AAtNDE.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CUq4o-LU8AAtNDE.jpg',
    'sizes': {'large': {'h': 762, 'resize': 'fit', 'w': 737},
     'medium': {'h': 620, 'resize': 'fit', 'w': 600},
     'small': {'h': 351, 'resize': 'fit', 'w': 340},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 669550509309583360,
    'source_status_id_str': '669550509309583360',
    'source_user_id': 3247919504,
    'source_user_id_str': '3247919504',
    'type': 'photo',
    'url': 'https://t.co/o25Mhbs

In [78]:
thanksgiving_df

,text
0,No place like home. #Thanksgiving #Family #Fai...
1,A 1904 Vegetarian Thanksgiving Dinner https://...
2,I'm proud to that I've quite a bit of the Than...
3,.@bonappetit We didn't think anyone else was a...
4,OK let's see them! Post pictures of that big t...
5,RT @frandrescher: https://t.co/no18fHFuvV Indu...
6,RT @cancerschmancer: https://t.co/h71sHgT4iw I...
7,With all the #thanksgiving #food going to be c...
8,RT @auntyaciduk: Happy Thanksgiving Eve! I can...
9,RT @chefcolton: Thanksgiving desserts ideas ea...
